In [1]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from matplotlib import pyplot as plt

In [2]:
model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

In [4]:

def apply_style_transfer():
    content_path = content_entry.get()
    style_path = style_entry.get()
    if not content_path or not style_path:
        return

    content_image = load_image(content_path)
    style_image = load_image(style_path)

    stylized_image = model(tf.constant(content_image), tf.constant(style_image))[0]

    # Display the stylized image
    stylized_img = Image.fromarray(np.uint8(np.squeeze(stylized_image) * 255))
    stylized_img = stylized_img.resize((400, 400))
    stylized_img = ImageTk.PhotoImage(stylized_img)

    result_label.config(image=stylized_img)
    result_label.image = stylized_img

# Function to reset displayed images
def reset_images():
    content_entry.delete(0, tk.END)
    style_entry.delete(0, tk.END)
    result_label.config(image=None)

# Function to open a file dialog for image selection
def open_image(entry_widget):
    file_path = filedialog.askopenfilename()
    entry_widget.delete(0, tk.END)
    entry_widget.insert(0, file_path)

# Function to load and preprocess an image
def load_image(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = img[tf.newaxis, :]
    return img

# Create the Tkinter window
root = tk.Tk()
root.title("Style Transfer Application")

# Create labels and entry fields for content and style images
content_label = tk.Label(root, text="Content Image:")
content_label.pack()
content_entry = tk.Entry(root)
content_entry.pack()
content_button = tk.Button(root, text="Open Content Image", command=lambda: open_image(content_entry))
content_button.pack()

style_label = tk.Label(root, text="Style Image:")
style_label.pack()
style_entry = tk.Entry(root)
style_entry.pack()
style_button = tk.Button(root, text="Open Style Image", command=lambda: open_image(style_entry))
style_button.pack()

# Create a button for style transfer
transfer_button = tk.Button(root, text="Apply Style Transfer", command=apply_style_transfer)
transfer_button.pack()

# Create a "Reset" button
reset_button = tk.Button(root, text="Reset", command=reset_images)
reset_button.pack()

# Create a label to display the stylized image
result_label = tk.Label(root)
result_label.pack()

root.mainloop()
